In [1]:
import os
import pandas as pd
import pandasql as ps
import seaborn as sns
import matplotlib.pyplot as plt
from my_s3_func import *
from func.df_fix_str_cols_to_dtime_and_conv import *

In [2]:
# my s3 global variables
s_mybucket = "jozsi-chicago-taxi-bb"
pathdir_proc_taxi = 'raw_data/to_process/taxi_data/'
pathdir_processed_taxi = 'raw_data/processed/taxi_data/'
pathdir_transf_taxi = 'transfomed_data/taxi_data/'
pathdir_proc_wheater = 'raw_data/to_process/weather_data/'
pathdir_processed_wheater = 'raw_data/processed/weather_data/'
pathdir_transf_wheater = 'transfomed_data/weather_data/'
pathdir_pay_type_master = 'transfomed_data/payment_type/'
pathdir_company_master = 'transfomed_data/company/'
pathdir_prev_masters = 'transfomed_data/master_table_previous_version/'
pathdir_areas =  'transfomed_data/community_areas/'
pathdir_date = 'transfomed_data/date/'

s_pay_types = 'payment_types.csv'
s_companies = 'companies.csv'
s_taxi_data = 'taxi_data.csv'
s_wheater_data = 'wheater_data.csv'
s_areas = 'community_areas.csv'
s_date = 'date.csv'

s_aws_id = os.getenv('AWS_ACCESS_ID')
s_aws_key = os.getenv('AWS_SEC_KEY')


### get data from s3

In [ ]:
# single files
# def object list
ls_path_file = [
    [pathdir_areas,s_areas],
    [pathdir_company_master, s_companies],
    [pathdir_date, s_date],
    [pathdir_pay_type_master, s_pay_types],
    ]

# get files from s3, a load to dynamic generate global dataframe variables
for s_path, s_file in ls_path_file:
    globals()['df_'+s_file.replace('.csv', '')] = load_s3_csv_to_df(s_bucket=s_mybucket, path_file=s_path+s_file, s_access_id=s_aws_id, s_sec_key=s_aws_key )
    print(f'{s_file} has been loaded to df_{s_file.replace('.csv', '')}')


In [ ]:
# load & concat all taxi_data
df_taxi_data_concat = load_s3_dir_csv_files(
    s_bucket=s_mybucket, 
    path_work=pathdir_transf_taxi,  
    s_access_id=s_aws_id, 
    s_sec_key=s_aws_key
    )


In [ ]:
# load & concat all wheater_data
df_wheather_data_concat = load_s3_dir_csv_files(
    s_bucket=s_mybucket, 
    path_work=pathdir_transf_wheater,  
    s_access_id=s_aws_id, 
    s_sec_key=s_aws_key
    )

### enrichments

In [6]:
# I always use a left join for this step to avoid losing data from the main table due to a join error
df_taxi_data_full = pd.merge(df_taxi_data_concat, df_community_areas, left_on='pickup_community_area_id', right_on='area code', how='left')
df_taxi_data_full.rename(columns={'community name': 'pickup_area_name'}, inplace=True)
df_taxi_data_full = pd.merge(df_taxi_data_full, df_community_areas, left_on='dropoff_community_area_id', right_on='area code', how='left')
df_taxi_data_full.rename(columns={'community name': 'dropof_area_name'}, inplace=True)
df_taxi_data_full = pd.merge(df_taxi_data_full, df_payment_types, on='payment_type_id', how='left')
df_taxi_data_full = pd.merge(df_taxi_data_full, df_companies, on='company_id', how='left')
df_taxi_data_full = pd.merge(df_taxi_data_full, df_wheather_data_concat, left_on='datetime_for_weather', right_on='datetime', how='left')


In [7]:
# preparation and join df_date
# fix values (if possible) and convert cols type to datatime
s_columns = ['trip_start_timestamp', 'trip_end_timestamp']
df_fix_str_cols_to_dtime_and_conv(df_taxi_data_full, s_columns)
# make a new "date" col for join
df_taxi_data_full['trip_start_date'] = df_taxi_data_full['trip_start_timestamp'].dt.date.astype(str)
df_taxi_data_full = pd.merge(df_taxi_data_full, df_date, left_on='trip_start_date', right_on='Date', how='left')


In [8]:
# dropping unnecessary columns
df_taxi_data_full = df_taxi_data_full.drop(columns=['area code_x','area code_y', 'datetime', 'Date'], errors='ignore')


In [9]:
# ''' 
# 1. Which 10 companies make the most money?
# Use two columns: Company name and the sum of the fare. Order by descending order.
# Make sure that the sum of the fares (total_fare) is not in scientific notation.
# '''

# pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',', ' ')

# query = '''
# SELECT 
# 	company, 
#     ROUND(SUM(trip_total),0) AS trips_total_usd
# FROM df_taxi_data_full
# GROUP BY company_id
# ORDER BY trips_total_usd DESC
# LIMIT 10
# '''
# df_res = ps.sqldf(query, locals())
# df_res


In [11]:
# '''
# 2. Show the 10 pickup community areas with the most rides.
# Use two columns: community area name and count of rides per area, in descending order for
# the rides.
# '''

# query = '''
# SELECT 
#     pickup_area_name,
#     COUNT(*) AS rides_count
# FROM df_taxi_data_full
# GROUP BY pickup_community_area_id
# ORDER BY rides_count DESC
# LIMIT 10
# '''
# df_res = ps.sqldf(query, locals())
# df_res

In [12]:
# '''
# 3. Get the count of taxi rides per day of week. In other words, we'd like to see which day has
# the most rides.
# Use the names of the days instead of numbers (Monday = 1, Tuesday = 2, etc.).
# HINT: Check CASE WHEN statements for replacing names of days. For ordering the results by
# the day you can use MIN(day_of_the_week).
# '''

# query = '''
# SELECT 
#     CASE 
#         WHEN day_of_week = '1' THEN 'Monday'
#         WHEN day_of_week = '2' THEN 'Tuesday'
#         WHEN day_of_week = '3' THEN 'Wednesday'
#         WHEN day_of_week = '4' THEN 'Thursday'
#         WHEN day_of_week = '5' THEN 'Friday'
#         WHEN day_of_week = '6' THEN 'Saturday'
#         WHEN day_of_week = '7' THEN 'Sunday'
#         ELSE 'wrong data'
#     END AS day_of_week,
#     COUNT(*) AS rides_count
# FROM df_taxi_data_full
# GROUP BY day_of_week
# ORDER BY rides_count DESC
# '''
# df_res = ps.sqldf(query, locals())
# df_res

### dirty, anomality cheks and notation

In [13]:

df_taxi_vis = df_taxi_data_full

# error detection function
def detect_errors(row):
    errors = []
    
    # missing data
    if row.isnull().any():
        errors.append('missing_data')
    
    # trip distance = 0, but the pickup and dropoff coordinates are diferent
    if(
        row['trip_miles'] == 0.0
        and (
            row['pickup_centroid_latitude'] != row['dropoff_centroid_latitude']
            or row['pickup_centroid_longitude'] != row['dropoff_centroid_longitude']
            )
        ):
        errors.append('is_movement')
    
    # trip time the calculated trip time are differ significantly
    timestamp_diff = abs(row['trip_end_timestamp'] - row['trip_start_timestamp']).seconds
    if(
        timestamp_diff - row['trip_seconds'] >= 900 
        ): # max 15 minutes granulation difference
        errors.append('trip_time')
    
    # was there a real fee payment?
    if(
        row['payment_type_id'] in [3,5,7] # 3: no charge, 4: dispute, 6: unknown (row['trip_total'] > 0.0 and )
        ):
        errors.append('is_payment')

    # there was'nt real trip, but there was fee payment
    if(
        row['trip_miles'] == 0.0
        and row['trip_total'] > 0.0
        ):
        errors.append('is_real_trip')
    
    return errors

# new column for the error types
df_taxi_vis['anomaly_types'] = df_taxi_vis.apply(detect_errors, axis=1)

# apply the error types to the dataframe
df_taxi_vis['anomaly_types'] = df_taxi_vis['anomaly_types'].apply(lambda x: x if x else None)



### Data visualizations with diagrams and graphs

In [14]:
# Exclude rows with anomalies for further analysis
df_taxi_vis_clean = df_taxi_vis[df_taxi_vis['anomaly_types'].isnull()]
# Date period
start_date = df_taxi_vis_clean['trip_start_timestamp'].min().strftime('%Y-%m-%d')
end_date = df_taxi_vis_clean['trip_end_timestamp'].max().strftime('%Y-%m-%d')
s_period = f'Period: {start_date} - {end_date}'


In [ ]:
# anomaly types by taxi companies
df_anomaly_counts = df_taxi_vis.explode('anomaly_types').groupby(['anomaly_types', 'company']).size().reset_index(name='count')
df_anomaly_counts = (
    df_anomaly_counts.groupby('anomaly_types', group_keys=False)
    .apply(lambda x: x.nlargest(5, 'count'))
    .reset_index(drop=True)
)

plt.figure(figsize=(12, 5))
sns.barplot(data=df_anomaly_counts, x='anomaly_types', y='count', hue='company', width=0.6)
plt.title('Top 5 dirty data types by taxi companies - ' + s_period)
plt.show()


In [ ]:
# Taxi companies with the highest revenue
df_taxi_revenue_top = df_taxi_vis_clean.groupby('company')['trip_total'].sum().nlargest(10).reset_index()
# total revenue
total_revenue = df_taxi_vis_clean['trip_total'].sum()
# percentage of total revenue
df_taxi_revenue_top['percentage'] = (df_taxi_revenue_top['trip_total'] / total_revenue) * 100

plt.figure(figsize=(12, 5))
plt.bar(df_taxi_revenue_top['company'], df_taxi_revenue_top['percentage'], width=0.3)
plt.title('Taxi companies with the highest revenue rate\n' + s_period)
plt.xticks(rotation=55)
plt.ylabel('Percentage of total revenue')
plt.xlabel('Taxi companies')

plt.show()


In [ ]:
# payment types distribution
df_payment_counts = df_taxi_vis_clean.groupby(['payment_type']).size().reset_index(name='count')
sum_payment_counts = df_payment_counts['count'].sum()
df_payment_counts['percentage'] = (df_payment_counts['count'] / sum_payment_counts * 100).round().astype(str) + '%'

plt.figure(figsize=(14, 8))
plt.title('Payment types distribution - ' + s_period)
plt.pie(df_payment_counts['count'], labels = df_payment_counts['percentage'])
plt.legend(df_payment_counts['payment_type'], title='Payment types', loc='upper left')
plt.show()


In [ ]:
# taxi trips by hour
df_taxi_vis_clean = df_taxi_vis_clean.copy()
df_taxi_vis_clean['hour'] = df_taxi_vis_clean['trip_start_timestamp'].dt.hour
df_hourly_rides = df_taxi_vis_clean.groupby(['hour']).size().reset_index(name='count')
plt.figure(figsize=(14, 8))
plt.bar(df_hourly_rides['hour'], df_hourly_rides['count'])
plt.title('Taxi trip starts by hour - ' + s_period)
plt.ylabel('Trip counts', fontsize=14)
plt.xlabel('Hours', fontsize=14, labelpad=20)
plt.xticks(range(0, 24))
plt.show()

In [ ]:

# taxi trips by day
df_daily_trip_cou = df_taxi_vis_clean.groupby(['trip_start_date']).size().reset_index(name='count')
df_daily_trip_cou['date_weak_day'] = df_daily_trip_cou['trip_start_date'] + ' - ' + pd.to_datetime(df_daily_trip_cou['trip_start_date']).dt.day_name()

plt.figure(figsize=(14, 8))
plt.bar(df_daily_trip_cou['date_weak_day'], df_daily_trip_cou['count'], width=0.5)
plt.title('Taxi trip starts by hour - ' + s_period)
plt.xticks(rotation=90)
plt.ylabel('Trip counts', fontsize=14)
plt.xlabel('Start date and day', fontsize=14, labelpad=20)
plt.show()